In [1]:
import pandas as pd
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import shift
from sklearn.metrics import r2_score as reg_acc
from sklearn.metrics import mean_squared_error as mse
from sklearn.neural_network import MLPRegressor as NN
from statsmodels.tsa.arima_model import ARIMA
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
features=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/features.csv')

In [0]:
stores=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/stores.csv')

In [0]:
train=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/train.csv')

In [0]:
merged_1=train.merge(stores, on=['Store'], how='left')

In [0]:
train=merged_1.merge(features, on=['Store','Date','IsHoliday'], how='left')

In [0]:
nan_values=train.isnull()
nan_values_count=[0]*train.shape[1]
for i in range(train.shape[1]):
  nan_values_count[i]=sum(nan_values.iloc[:,i])

In [0]:
train['MarkDown1']=train['MarkDown1'].fillna(0)
train['MarkDown2']=train['MarkDown2'].fillna(0)
train['MarkDown3']=train['MarkDown3'].fillna(0)
train['MarkDown4']=train['MarkDown4'].fillna(0)
train['MarkDown5']=train['MarkDown5'].fillna(0)

In [0]:
train['month']=pd.DatetimeIndex(train['Date']).month
train['day']=pd.DatetimeIndex(train['Date']).day
train['Type']=train['Type'].astype('category')
train['StoreType']=train['Type'].cat.codes
train=train.drop('Type',axis=1)
train['date']=train['Date']
train=train.drop('Date',axis=1)

In [0]:
test=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/test.csv')

In [0]:
merged_1=test.merge(stores, on=['Store'], how='left')
test=merged_1.merge(features, on=['Store','Date','IsHoliday'], how='left')
nan_values=test.isnull()
nan_values_count=[0]*test.shape[1]
for i in range(test.shape[1]):
  nan_values_count[i]=sum(nan_values.iloc[:,i])
test['MarkDown1']=test['MarkDown1'].fillna(0)
test['MarkDown2']=test['MarkDown2'].fillna(0)
test['MarkDown3']=test['MarkDown3'].fillna(0)
test['MarkDown4']=test['MarkDown4'].fillna(0)
test['MarkDown5']=test['MarkDown5'].fillna(0)
test['CPI']=test['CPI'].fillna(np.mean(test['CPI']))
test['Unemployment']=test['Unemployment'].fillna(np.mean(test['Unemployment']))
test['month']=pd.DatetimeIndex(test['Date']).month
test['day']=pd.DatetimeIndex(test['Date']).day
test['Type']=test['Type'].astype('category')
test['StoreType']=test['Type'].cat.codes
test=test.drop('Type',axis=1)
test['date']=test['Date']
test=test.drop('Date',axis=1)

In [0]:
train['sales']=train['Weekly_Sales']
train=train.drop('Weekly_Sales',axis=1)

In [14]:
print(train.shape)
print(test.shape)

(421570, 18)
(115064, 17)


In [0]:
#!pip3 install torch torchvision
#!pip3 install torch --upgrade

In [16]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [0]:
store1=sum(train['Store']==1)
store1=10000

In [0]:
#train=train.iloc[:store1,:]

In [0]:
train_x=train.iloc[: int(0.8*train.shape[0]) , 0:16].as_matrix()
train_y=train.iloc[: int(0.8*train.shape[0]) , 17].as_matrix()
test_x=train.iloc[ int(0.8*train.shape[0]) :, 0:16].as_matrix()
test_y=train.iloc[ int(0.8*train.shape[0]) :, 17].as_matrix()

In [20]:
scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_x.fit(train_x)
scaler_y = np.max(train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
train_x=scaler_x.transform(train_x)
train_y=train_y/scaler_y
test_x=scaler_x.transform(test_x)
test_y=test_y/scaler_y

In [0]:
time_stamps=1

In [0]:
train_index=train_x.shape[0]/time_stamps
train_index=int(train_index)
train_index=train_index*time_stamps

In [0]:
test_index=test_x.shape[0]/time_stamps
test_index=int(test_index)
test_index=test_index*time_stamps

In [0]:
train_x=train_x[:train_index]
train_y=train_y[:train_index]
test_x=test_x[:test_index]
test_y=test_y[:test_index]

In [26]:
train_x.shape

(337256, 16)

In [0]:
train_x=train_x.reshape(-1,time_stamps,train_x.shape[1])
test_x=test_x.reshape(-1,time_stamps,test_x.shape[1])
train_y=train_y.reshape(-1,time_stamps)
test_y=test_y.reshape(-1,time_stamps)

In [28]:
test_x.shape

(84314, 1, 16)

In [0]:
model = Sequential()
model.add(LSTM(512,activation='relu',return_sequences=True,input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(time_stamps,activation='relu'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [0]:
model.fit(train_x, train_y, epochs=50, batch_size=100, validation_data=(test_x, test_y), shuffle=False)

Train on 337256 samples, validate on 84314 samples
Epoch 1/50
337256/337256 [==============================] - 68s 202us/step - loss: 0.0012 - val_loss: 6.9917e-04
Epoch 2/50
337256/337256 [==============================] - 66s 197us/step - loss: 0.0011 - val_loss: 7.0669e-04
Epoch 3/50
337256/337256 [==============================] - 66s 196us/step - loss: 0.0011 - val_loss: 6.7406e-04
Epoch 4/50
337256/337256 [==============================] - 66s 196us/step - loss: 0.0011 - val_loss: 6.6944e-04
Epoch 5/50
337256/337256 [==============================] - 66s 195us/step - loss: 0.0010 - val_loss: 6.7886e-04
Epoch 6/50
337256/337256 [==============================] - 66s 196us/step - loss: 0.0010 - val_loss: 6.5868e-04
Epoch 7/50
337256/337256 [==============================] - 66s 195us/step - loss: 0.0010 - val_loss: 6.9452e-04
Epoch 8/50
337256/337256 [==============================] - 66s 195us/step - loss: 9.7023e-04 - val_loss: 7.0121e-04
Epoch 9/50
337256/337256 [===============

In [0]:
prediction=model.predict(test_x)

In [0]:
prediction=prediction*scaler_y

In [0]:
prediction=prediction.reshape(-1)
test_y=test_y.reshape(-1)
indices=np.arange(prediction.shape[0])
plt.plot(indices,test_y*scaler_y,'b',indices,prediction,'r')

In [0]:
reg_acc(test_y*scaler_y,prediction)

In [0]:
mse(test_y*scaler_y,prediction)

In [0]:
temp=test.iloc[:, 0:15].as_matrix()

In [0]:
temp=scaler_x.transform(temp)

In [0]:
temp=temp.reshape(-1,1,15)

In [0]:
test_prediction=model.predict(temp)

In [0]:
test_prediction=test_prediction*scaler_y

In [0]:
test_prediction

In [0]:
submission=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/sampleSubmission.csv')

In [0]:
submission['Weekly_Sales']=test_prediction

In [0]:
submission.to_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/kaggle_submission.csv',index=False)

In [0]:
submission.head()

In [0]:
sum(train['Store']==1)

In [0]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [0]:
train.shape

In [0]:
train.head()